In [1]:
# Cell 1: Imports
import pdfplumber
import csv
import re
import pandas as pd
from typing import List, Dict, Tuple

In [2]:
# Cell 2: Helper functions
def clean_text(text: str) -> str:
    """Clean and normalize text content."""
    if not text:
        return ""
    # Remove extra whitespace and normalize spaces
    text = re.sub(r'\s+', ' ', text.strip())
    # Remove any form feeds or other special characters
    text = text.replace('\f', ' ')
    return text

In [3]:
def process_table(table, current_mcc=None, current_title=None, 
                 current_description=None, current_included=None, 
                 current_similar=None) -> Tuple[str, str, str, str, str, List[Dict]]:
    """Process a single table and return current state and completed records."""
    mcc_data = []
    
    for row_idx, row in enumerate(table):
        if not row or all(cell is None or str(cell).strip() == "" for cell in row):
            continue
            
        # Clean row data and remove empty columns
        row = [clean_text(str(cell)) for cell in row if cell is not None]
        if not row:  # Skip if row is empty after cleaning
            continue
            
        print(f"Processing row {row_idx}: {row}")  # Debug print
        
        # Check if this is a header row
        if any(header in str(row).upper() for header in ["MCC TITLE", "MCC DESCRIPTION"]):
            print("Skipping header row")  # Debug print
            continue
        
        # First row of entry should contain MCC code and title
        if any(cell.isdigit() and len(cell) == 4 for cell in row):
            # If we have complete previous record, save it
            if all([current_mcc, current_title]):
                record = {
                    "MCC": current_mcc,
                    "MCC_TITLE": current_title,
                    "MCC_DESCRIPTION": current_description or "",
                    "INCLUDED_IN_MCC": current_included or "",
                    "SIMILAR_MERCHANTS": current_similar or ""
                }
                print(f"Saving record: {record}")  # Debug print
                mcc_data.append(record)
            
            # Find MCC code and title
            non_empty_cells = [cell for cell in row if cell.strip()]
            for i, cell in enumerate(non_empty_cells):
                if cell.isdigit() and len(cell) == 4:
                    current_mcc = cell
                    # Title should be the next non-empty cell
                    if i + 1 < len(non_empty_cells):
                        current_title = non_empty_cells[i + 1]
                    break
            
            current_description = None
            current_included = None
            current_similar = None
            print(f"Started new record: MCC={current_mcc}, Title={current_title}")  # Debug print
            
        # Description row (contains description text and possibly included/similar items)
        elif current_mcc and any("Merchants classified with this MCC" in str(cell) for cell in row):
            non_empty_cells = [cell for cell in row if cell.strip()]
            
            # Find description (cell containing the marker text)
            description_cell = next(cell for cell in non_empty_cells 
                                 if "Merchants classified with this MCC" in str(cell))
            current_description = description_cell
            
            # Look for included items and similar merchants in remaining cells
            remaining_cells = [cell for cell in non_empty_cells if cell != description_cell]
            for cell in remaining_cells:
                # Similar merchants column contains MCC codes (XXXX – Description)
                if any(re.search(r'\d{4}\s*[–-]', line) for line in cell.split('\n')):
                    current_similar = cell
                else:
                    current_included = cell
            
            print(f"Added included items: {current_included}")  # Debug print
            print(f"Added similar merchants: {current_similar}")  # Debug print
            
        # Handle overflow rows (continuing content from previous row)
        elif current_mcc and current_description and not any(cell.strip().isdigit() and len(cell.strip()) == 4 for cell in row):
            non_empty_cells = [cell for cell in row if cell.strip()]
            for cell in non_empty_cells:
                # Skip empty cells
                if not cell.strip():
                    continue
                    
                # Check if this is description overflow (typically a continuation of a sentence)
                if not cell.strip().startswith('•') and not any(re.search(r'\d{4}\s*[–-]', line) for line in cell.split('\n')):
                    # If it starts with lowercase or continues a sentence, it's likely description overflow
                    first_word = cell.strip().split()[0]
                    if first_word[0].islower() or not current_description.strip().endswith('.'):
                        current_description = (current_description or "") + " " + cell
                        print(f"Added overflow to description")  # Debug print
                        continue
                
                # Check if this is similar merchants overflow (contains MCC codes)
                if any(re.search(r'\d{4}\s*[–-]', line) for line in cell.split('\n')):
                    current_similar = (current_similar or "") + "\n" + cell
                    print(f"Added overflow to similar merchants")  # Debug print
                # Otherwise assume it's included items overflow
                else:
                    current_included = (current_included or "") + "\n" + cell
                    print(f"Added overflow to included items")  # Debug print

    # Return current state and completed records
    # Note: We don't save the last record here anymore - that's handled at the PDF level
    return current_mcc, current_title, current_description, current_included, current_similar, mcc_data

In [4]:
def extract_mcc_data(pdf_path: str) -> List[Dict]:
    """Process entire PDF and return extracted MCC records."""
    all_mcc_data = []
    current_mcc = None
    current_title = None
    current_description = None
    current_included = None
    current_similar = None
    
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, 1):
            print(f"\nProcessing page {page_num}")
            tables = page.extract_tables()
            print(f"Found {len(tables)} tables on page {page_num}\n")
            
            for table_num, table in enumerate(tables, 1):
                print(f"Processing table {table_num} on page {page_num}")
                
                # Process table with current state
                current_mcc, current_title, current_description, current_included, current_similar, records = process_table(
                    table, 
                    current_mcc=current_mcc,
                    current_title=current_title,
                    current_description=current_description,
                    current_included=current_included,
                    current_similar=current_similar
                )
                
                all_mcc_data.extend(records)
                
    # Don't forget to save the final record
    if all([current_mcc, current_title]):
        record = {
            "MCC": current_mcc,
            "MCC_TITLE": current_title,
            "MCC_DESCRIPTION": current_description or "",
            "INCLUDED_IN_MCC": current_included or "",
            "SIMILAR_MERCHANTS": current_similar or ""
        }
        all_mcc_data.append(record)
        
    return all_mcc_data

In [5]:
# Cell 5: CSV saving function
def save_to_csv(data: List[Dict], output_path: str) -> None:
    """Save extracted data to CSV file."""
    if not data:
        print("No data to save")
        return
        
    try:
        fieldnames = ["MCC", "MCC_TITLE", "MCC_DESCRIPTION", "INCLUDED_IN_MCC","SIMILAR_MERCHANTS"]
        with open(output_path, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(data)
        print(f"Data successfully saved to {output_path}")
    
    except Exception as e:
        print(f"Error saving CSV: {str(e)}")

In [ ]:
# Cell 6: Test with a single page/table
# You can run this cell to test with a specific page or table
pdf_path = "visa-merchant-data-standards-manual.pdf"
mcc_data = extract_mcc_data(pdf_path)

# Print first few records to verify
print("\nFirst few records:")
for record in mcc_data[:3]:
    print(record)

In [ ]:
mcc_df = pd.DataFrame(mcc_data)
save_to_csv(mcc_data,"mcc_visa_pdf_data.csv")


In [ ]:
dict(mcc_df.iloc[5])